**PG54469**, **Edgar Wchua Pires Guilherme**, Mestrado em Engenharia Informática

**MFES** - Métodos Formais em Engenharia de Software



---
**Ficha 1**

**Modelação em Lógica Proposicional**

https://haslab.github.io/MFES/2324/ficha-1.pdf





---

**Instalação das bibliotecas**

In [1]:
!pip install z3-solver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 10.3 MB/s eta 0:00:00


---

**Exercício 1 - Configuração de produtos**


1. Codificar as dependências entre configurações como uma fórmula proposicional ψ.
2. Codificar as opções selecionadas pelo cliente como uma fórmula proposicional φ.
3. Usar o SAT solver para verificar se ψ ∧ φ não é contraditório.
Considere agora a seguinte dependência entre as configuraçõoes disponíveis para a personalização de um automóvel:

“O ar condicionado **Thermotronic comfort** *requer* uma **bateria de alta capacidade**,
*exceto* quando *combinado* com **motores a gasolina de 3,2 litros**.”

Será que um cliente pode *escolher* o ar condicionado **Thermotronic comfort**, uma **bateria de pequena capacidade**, mas *não escolher* o **motor de 3,2 litros**? Como poderia responder a esta pergunta com a ajuda de um SAT solver?

In [10]:
from z3 import *

thermotronic_comfort = Bool('Thermotronic comfort')
bateria_alta_capacidade = Bool("bateria de alta capacidade")
bateria_pequena_capacidade = Bool("bateria de pequena capacidade")
motor_gasolina_3_2 = Bool("motores a gasolina de 3,2 litros")

s = Solver()

# O ar condicionado Thermotronic comfort requer uma bateria de alta capacidade
# exceto quando combinado com motores a gasolina de 3,2 litros
s.add(Implies(And(thermotronic_comfort, Not(motor_gasolina_3_2)), bateria_alta_capacidade))

s.add(Implies(bateria_alta_capacidade, Not(bateria_pequena_capacidade)))

print('Teste da escolha do cliente')
s.push()

s.add(thermotronic_comfort)
s.add(bateria_pequena_capacidade)
s.add(Not(motor_gasolina_3_2))

r = s.check()

if r == sat:
    print(s.model())
    print('O cliente pode escolher o ar condicionado Thermotronic comfort, uma bateria de pequena capacidade, mas não escolher o motor de 3,2 litros')
else:
    print('O cliente não pode escolher o ar condicionado Thermotronic comfort, uma bateria de pequena capacidade, mas não escolher o motor de 3,2 litros')

s.pop()


Teste da escolha do cliente
O cliente não pode escolher o ar condicionado Thermotronic comfort, uma bateria de pequena capacidade, mas não escolher o motor de 3,2 litros


---

**Exercício 2 - Distribuição de gabinetes**

Considere que temos 3 gabinetes e queremos
distribuir 4 pessoas (Ana=1, Nuno=2, Pedro=3 e Maria=4) por esses gabinetes.

Considere que foram estipuladas as seguintes regras de ocupação dos gabinetes:

(a) Cada pessoa ocupa um único gabinete.

(b) O Nuno e o Pedro não podem partilhar gabinete.

(c) Se a Ana ficar sozinha num gabinete, então o Pedro também terá que ficar sozinho num gabinete.

(d) Cada gabinete só pode acomodar, no máximo, 2 pessoas.


In [23]:
gabinetes = [1, 2, 3]
pessoas = ["Ana", "Nuno", "Pedro", "Maria"]
n_pessoas = len(pessoas)
n_gabinetes = len(gabinetes)

x = {}
for p in pessoas:
  x[p] = {}
  for g in gabinetes:
    x[p][g] = Bool("%s - %s" % (p, g))

s = Solver()

# Cada pessoa ocupa pelo menos um gabinete.
for p in pessoas:
  s.add(Or([x[p][g] for g in gabinetes]))


# Cada pessoa ocupa um único gabinete
# for p in pessoas:
#   for i in range(n_gabinetes - 1):
#     for j in range(i + 1, n_gabinetes):
#       s.add(Implies(x[p][i], Not(x[p][j])))
for p in pessoas:
  for g in gabinetes:
    s.add([Implies(x[p][g], Not(x[p][j])) for j in gabinetes if j > g])


# O Nuno e o Pedro não podem partilhar gabinete
def nao_pode_partilhar_gabinete(p1, p2):
    return And([Implies(x[p1][g], Not(x[p2][g])) for g in gabinetes])

s.add(nao_pode_partilhar_gabinete("Nuno", "Pedro"))

# Se a Ana ficar sozinha num gabinete, então o Pedro também terá que ficar sozinho num gabinete
def fica_sozinho_no_gabinete(p):
    return And([Implies(x[p][g], And([Not(x[i][g]) for i in pessoas if i != p])) for g in gabinetes])

s.add(Implies(fica_sozinho_no_gabinete("Ana"), fica_sozinho_no_gabinete("Pedro")))

# Cada gabinete só pode acomodar, no máximo, 2 pessoas.
def maximo_2_pessoas_por_gabinete(g):
  for a in pessoas:
    for b in [i for i in pessoas if i != a]:
        s.add(Implies(And(x[a][g], x[b][g]), And([Not(x[c][g]) for c in pessoas if c not in [a, b]])))

for g in gabinetes:
  maximo_2_pessoas_por_gabinete(g)

# Solve
if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabinetes:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
else:
  print("Não tem solução.")

Ana fica no gabinete 2
Nuno fica no gabinete 2
Pedro fica no gabinete 1
Maria fica no gabinete 3
